# TAREA 3 - ETL DATASET BI

## INTEGRANTES:
- Apolonio Cuevas Manuel Alberto
- Sansores Arjona Alejandro Jesus
- Israel Alejandro Cel Alcocer
- Salvador Iram Salas Baez
- Leo Sebastian Contreras Raymund

 ## Configuracion del Ambiente


In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session

session = get_active_session()
print(session)


### Automatizacion de la carga en kaggle

## Cargar la tabla desde el csv usando pandas para pasar el encoding latin-1

In [ ]:
# Importa pandas, lo vamos a necesitar como un paso intermedio
import pandas as pd
import snowflake.snowpark.types as T
import snowflake.snowpark.functions as F
from snowflake.snowpark.types import StructType, StructField, IntegerType, StringType, DoubleType

table_name = "TAREA3_RAW"

# 1. El esquema sigue igual
schema = StructType([
    StructField("fNAME", StringType()),
    StructField("lNAME", StringType()),
    StructField("Age", IntegerType()),
    StructField("gender", StringType()),
    StructField("country", StringType()),
    StructField("residence", StringType()),
    StructField("entryEXAM", DoubleType()),
    StructField("prevEducation", StringType()),
    StructField("studyHOURS", IntegerType()),
    StructField("Python", DoubleType()),
    StructField("DB", DoubleType()),
])

# 2. Abre el archivo localmente con el open() normal de Python
with open("TSID-TAREA3/bi.csv", 'r', encoding='latin-1') as f:
    pandas_df = pd.read_csv(f, header=0, names=[field.name for field in schema.fields])

# 3. Convierte el DataFrame de Pandas a Snowpark
dataframe = session.create_dataframe(pandas_df, schema=None)

# 4. Guarda el DataFrame en la tabla final
dataframe.write.mode("overwrite").save_as_table(table_name)

print(f"Tabla '{table_name}' cargada correctamente.")
dataframe.show()

## Valores distintos de cada columna, comprobamos la inconsistencia de anotacion

In [ ]:
from snowflake.snowpark.functions import col
RAW_TABLE_NAME = "TAREA3_RAW"

raw_data = session.table(RAW_TABLE_NAME)
print("Valores distinctos de cada columna")

columns = ["GENDER", "PREVEDUCATION", "COUNTRY"]

for column in columns:
    print(column)
    print(raw_data.select(column).distinct().show())

## Corrección y estandarización de datos

In [ ]:
from snowflake.snowpark.functions import col, lower, when

RAW_TABLE_NAME = "TAREA3_RAW"
CLEANED_TABLE_NAME = "TAREA3_CLEANED"

# Leer la tabla original
raw_data = session.table(RAW_TABLE_NAME)

# Limpiar columna GENDER: pasar a minúsculas y corregir abreviaciones
cleaned_data = raw_data.with_column(
    "GENDER",
    when(lower(col("GENDER")).isin("m", "male"), "male")
    .when(lower(col("GENDER")).isin("f", "female"), "female")
    .otherwise(lower(col("GENDER")))
)

# Limpiar PREVEDUCATION: todo a minúsculas y corregir errores
cleaned_data = cleaned_data.with_column(
    "PREVEDUCATION",
    when(lower(col("PREVEDUCATION")).like("%high school%"), "high school")
    .when(lower(col("PREVEDUCATION")).like("%bachelor%") | lower(col("PREVEDUCATION")).like("%barrrchelors%"), "bachelors")
    .when(lower(col("PREVEDUCATION")).like("%master%"), "masters")
    .when(lower(col("PREVEDUCATION")).like("%doctor%"), "doctorate")
    .when(lower(col("PREVEDUCATION")).like("%diploma%"), "diploma")
    .otherwise(lower(col("PREVEDUCATION")))
)

# Limpiar COUNTRY: todo a minús culas y corregir abreviaciones o nombres alternativos
cleaned_data = cleaned_data.with_column(
    "COUNTRY",
    when(lower(col("COUNTRY")).like("norway") | lower(col("COUNTRY")).like("norge"), "norway")
    .when(lower(col("COUNTRY")).like("rsa"), "south africa")
    .otherwise(lower(col("COUNTRY")))
)


print(f"Tabla '{CLEANED_TABLE_NAME}' creada correctamente con los datos normalizados.")
cleaned_data.show()


## Identifiacion y manjeo de valores nulos

In [ ]:
from snowflake.snowpark.functions import col


RAW_TABLE_NAME = "TAREA3_RAW"
raw_data = session.table(RAW_TABLE_NAME)

text_columns = ["FNAME", "LNAME", "RESIDENCE", "GENDER", "PREVEDUCATION", "COUNTRY", "PYTHON", "DB"]

print("Buscando valores nulos...\n")

for column in text_columns:
    null_df = raw_data.filter(col(column).is_null())
    
    if null_df.count() > 0:
        print(f"--- Columna con nulos: {column} ---")
        
        # 3. Muestra las filas nulas encontradas.
        null_df.select(column).show()
        print("\n")

print("Búsqueda finalizada.")

## reemplazamos con -1

In [ ]:
from snowflake.snowpark.functions import col, when, lit

RAW_TABLE_NAME = "TAREA3_RAW"
CLEANED_TABLE_NAME = "TAREA3_CLEANED"

# Leer la tabla original
raw_data = session.table(RAW_TABLE_NAME)

# Reemplazar NULLs en la columna PYTHON por "NA"
cleaned_data = cleaned_data.with_column(
    "PYTHON",
    when(col("PYTHON").is_null(), -1 ).otherwise(col("PYTHON"))
)

print(f" Tabla '{CLEANED_TABLE_NAME}' creada correctamente con los datos normalizados.")
cleaned_data.show()

## Estudiantes con menos de 25 años

In [ ]:
cleaned_data.filter(col("AGE") < 25).show()

## estudiantes inscritos a nivel post universitario.

In [ ]:
query = cleaned_data.filter(col("PREVEDUCATION").isin("masters", "doctorate"))
query.show()
query.select("PREVEDUCATION").distinct().show()

## Subir la tabla limpia a la base de datos

In [ ]:
cleaned_data.write.mode("overwrite").save_as_table("TAREA3_CLEANED")

In [ ]:
---> set the Role
USE ROLE DESAROLLADOR;
---> set the Warehouse
USE WAREHOUSE SNOWFLAKE_LEARNING_WH;

USE DATABASE INFO_EMPLEADOS;

SELECT * 
FROM TAREA3_CLEANED